In [8]:
pip install openpyxl 

In [3]:
import numpy as np
import pandas as pd

bot_name = "CoronaBot"
# dataset coronavirus WHO
pd.set_option('max_colwidth', 100)  # Increase column width
data = pd.read_excel(r"C:\Users\amine\OneDrive\Bureau\ai-faqbot-who-master\WHO_FAQ.xlsx")
data.head(50)

,Context,Answer,Unnamed: 2
0,What is a coronavirus?,Coronaviruses are a large family of viruses which may cause illness in animals or humans.,NaN
1,What is a coronavirus?,"In humans, several coronaviruses are known to cause respiratory infections ranging from the comm...",NaN
2,What is COVID-19?,COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new ...,NaN
3,What are the symptoms of COVID-19?,"The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have...",NaN
4,What are the symptoms of COVID-19?,Some people become infected but don’t develop any symptoms and don't feel unwell. Most people (a...,NaN
5,What are the symptoms of COVID-19?,Around 1 out of every 6 people who gets COVID-19 becomes seriously ill and develops difficulty b...,NaN
6,What are the symptoms of COVID-19?,"Older people, and those with underlying medical problems like high blood pressure, heart problem...",NaN
7,How does COVID-19 spread?,People can catch COVID-19 from others who have the virus. The disease can spread from person to ...,NaN
8,How does COVID-19 spread?,People can also catch COVID-19 if they breathe in droplets from a person with COVID-19 who cough...,NaN
9,Can the virus that causes COVID-19 be transmitted through the air?,Studies to date suggest that the virus that causes COVID-19 is mainly transmitted through contac...,NaN


In [2]:
##### Use USE pretrained model to extract response encodings.
import tensorflow as tf
import tensorflow_hub as hub # TensorFlow Hub est un dépôt de modèles de machine learning entraînés
import tensorflow_text
import re #Ce module fournit des opérations de correspondance d'expressions régulières similaires à celles trouvées dans Perl
data = pd.read_excel(r"C:\Users\amine\OneDrive\Bureau\ai-faqbot-who-master\WHO_FAQ.xlsx")

def preprocess_sentences(input_sentences):

    return [re.sub(r'(covid-19|covid)', 'coronavirus', input_sentence, flags=re.I) 
            for input_sentence in input_sentences]
        
# Load module containing USE
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

# Create response embeddings
response_encodings = module.signatures['response_encoder'](
        input=tf.constant(preprocess_sentences(data.Answer)),
        context=tf.constant(preprocess_sentences(data.Context)))['outputs']

In [3]:
test_questions = [
    "What about pregnant women?",
    "Wat is de lengte van de incubatietijd?",
    "Are animals contagious COVID-19?",
    "Are there medicine against the coronavirus?",
    "Can I breastfead when I have COVID-19?",
    "Should I stay inside the house?",  # English questions are also possible.
    "C'est quoi le coronavirus ?"  
]

# Create encodings for test questions

question_encodings = module.signatures['question_encoder'](
    tf.constant(preprocess_sentences(test_questions))
)['outputs']

# Get the responses
test_responses = data.Answer[np.argmax(np.inner(question_encodings, response_encodings), axis=1)]

# Show them in a dataframe
pd.DataFrame({'Test Questions': test_questions, 'Test Responses': test_responses})


,Test Questions,Test Responses
65,What about pregnant women?,"due to changes in their bodies and immune systems, we know that pregnant women can be badly affe..."
44,Wat is de lengte van de incubatietijd?,"Most estimates of the incubation period for COVID-19 range from 1-14 days, most commonly around ..."
45,Are animals contagious COVID-19?,"Coronaviruses are a large family of viruses that are common in animals. Occasionally, people get..."
32,Are there medicine against the coronavirus?,"Not yet. To date, there is no vaccine and no specific antiviral medicine to prevent or treat COV..."
72,Can I breastfead when I have COVID-19?,Yes. Women with COVID-19 can breastfeed if they wish to do so.
17,Should I stay inside the house?,"Stay home if you feel unwell. If you have a fever, cough and difficulty breathing, seek medical ..."
45,C'est quoi le coronavirus ?,"Coronaviruses are a large family of viruses that are common in animals. Occasionally, people get..."
